# Gallery Example: Γ/M/1 Queue (Gamma Arrivals)

This example demonstrates a Γ/M/1 queueing system:
- **Arrivals**: Gamma distribution inter-arrival times
- **Service**: Exponential service times
- **Servers**: 1 server
- **Capacity**: Infinite
- **Scheduling**: FCFS

The Gamma distribution provides flexible modeling of arrival patterns with adjustable variance, generalizing both Exponential and Erlang distributions.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_gamm1(): Network {    """Create Γ/M/1 queueing model"""    model = Network("Gamma/M/1")        # Block 1: nodes    source = Source(model, "mySource")    queue = Queue(model, "myQueue", SchedStrategy.FCFS)    sink = Sink(model, "mySink")        # Block 2: classes    oclass = OpenClass(model, "myClass")    # Gamma arrivals with mean=1 and shape parameter (α=2, rate=2 gives mean=1)    source.setArrival(oclass, Gamma.fit_mean_and_order(1.0, 2))    # Exponential service with rate μ=2    queue.setService(oclass, Exp(2))        # Block 3: topology    P = model.initRoutingMatrix()    P.addRoute(oclass, source, queue, 1.0)    P.addRoute(oclass, queue, sink, 1.0)    model.link(P)        return model# Create the modelmodel = gallery_gamm1()

## Theoretical Analysis for Γ/M/1

For Γ/M/1 with:
- **Arrival process**: Gamma with mean=1, shape=2
- **Service rate**: μ = 2 (mean service time = 0.5)
- **Utilization**: ρ = λ/μ = 1/2 = 0.5

Gamma distribution characteristics:
- **Shape parameter**: α = 2
- **Rate parameter**: β = 2 (since mean = α/β = 2/2 = 1)
- **Variance**: α/β² = 2/4 = 0.5
- **Coefficient of Variation**: C²ₐ = 1/α = 1/2 = 0.5

This is between Exponential (C²=1) and Deterministic (C²=0).

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// MVA Solver
val solver_mva = MVA(model)
val avg_table_mva = solver_mva.avgTable
println("\nMVA Solver:")
println(avg_table_mva)
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)

In [ ]:
// Compare Gamma with Exponential and Erlang arrivals
println("\n=== Arrival Distribution Comparison ===")

fun create_arrival_comparison_models(): Network {
    """Create models with different arrival distributions"""
    val models = {}
// Exponential arrivals (C² = 1)
    val model_exp = Network("M/M/1")
    val source = Source(model_exp, "Source")
    val queue = Queue(model_exp, "Queue", SchedStrategy.FCFS)
    val sink = Sink(model_exp, "Sink")
    val oclass = OpenClass(model_exp, "Class")
    source.setArrival(oclass, Exp(1))  # Same mean arrival rate
    queue.setService(oclass, Exp(2))   # Same service rate
    val P = model_exp.initRoutingMatrix()
    P.addRoute(oclass, source, queue, 1.0)
    P.addRoute(oclass, queue, sink, 1.0)
    model_exp.link(P)
    models["Exponential (C²=1.0)"] = model_exp
// Erlang arrivals (C² = 0.5)
    val model_erl = Network("Er/M/1")
    val source = Source(model_erl, "Source")
    val queue = Queue(model_erl, "Queue", SchedStrategy.FCFS)
    val sink = Sink(model_erl, "Sink")
    val oclass = OpenClass(model_erl, "Class")
    source.setArrival(oclass, Erlang.fit_mean_and_order(1, 2))  # C² = 1/2 = 0.5
    queue.setService(oclass, Exp(2))
    val P = model_erl.initRoutingMatrix()
    P.addRoute(oclass, source, queue, 1.0)
    P.addRoute(oclass, queue, sink, 1.0)
    model_erl.link(P)
    models["Erlang-2 (C²=0.5)"] = model_erl
// Gamma arrivals (C² = 0.5, same as Erlang-2)
    models["Gamma α=2 (C²=0.5)"] = model
    
    return models

val comparison_models = create_arrival_comparison_models()

println("Arrival Distribution | C²   | Response Time | Queue Length")
println("-" * 60)

for dist_name, model_comp in comparison_models.items():
    val solver_comp = MVA(model_comp)
    val avg_table_comp = solver_comp.avgTable
    
    val resp_time = float(avg_table_comp.iloc[1, 2])
    val queue_length = float(avg_table_comp.iloc[1, 3])
// Extract CV² from distribution name
    val cv_sq = dist_name.split("C²=")[1].split(")")[0] if "C²=" in dist_name else "1.0"
    
    println(f"{dist_name:18s} | {cv_sq:4s} |     {resp_time:.4f}    |    {queue_length:.4f}")

println("\nObservations:")
println("1. Gamma and Erlang-2 give identical results (same C²)")
println("2. Lower variability (C² < 1) improves performance vs. Exponential")
println("3. Gamma distribution provides flexible modeling between Exponential and Erlang")

In [ ]:
// Analyze Gamma shape parameter effects
println("\n=== Gamma Shape Parameter Analysis ===")

fun create_gamma_shape_model(alpha): Network {
    """Create Γ/M/1 model with specified shape parameter"""
    val model_shape = Network(f"Gamma-α{alpha}/M/1")
    val source = Source(model_shape, "Source")
    val queue = Queue(model_shape, "Queue", SchedStrategy.FCFS)
    val sink = Sink(model_shape, "Sink")
    
    val oclass = OpenClass(model_shape, "Class")
    source.setArrival(oclass, Gamma.fit_mean_and_order(1.0, alpha))  # Mean=1, shape=α
    queue.setService(oclass, Exp(2))  # Same service rate
    
    val P = model_shape.initRoutingMatrix()
    P.addRoute(oclass, source, queue, 1.0)
    P.addRoute(oclass, queue, sink, 1.0)
    model_shape.link(P)
    
    return model_shape
// Test different shape parameters
val alpha_values = [0.5, 1, 2, 3, 5, 10]

println("Shape α | C² = 1/α | Response Time | Queue Length | Distribution")
println("-" * 65)

for alpha in alpha_values:
    try:
        val model_alpha = create_gamma_shape_model(alpha)
        val solver_alpha = MVA(model_alpha)
        val avg_table_alpha = solver_alpha.avgTable
        
        val resp_time = float(avg_table_alpha.iloc[1, 2])
        val queue_length = float(avg_table_alpha.iloc[1, 3])
        val cv_squared = 1.0 / alpha
// Special cases
        if alpha == 1:
            val dist_type = "Exponential"
        elif alpha == int(alpha) and alpha > 1:
            val dist_type = f"Erlang-{int(alpha)}"
        else:
            val dist_type = "Gamma"
        
        println(f"  {alpha:4.1f}  |  {cv_squared:.3f}   |     {resp_time:.4f}    |    {queue_length:.4f}     | {dist_type}")
        
    except Exception as e:
        println(f"  {alpha:4.1f}  |  {1.0/alpha:.3f}   |     Error     |     Error      | {str(e)[:10]}...")

println("\nShape Parameter Effects:")
println("• α < 1: Higher variability than Exponential (more bursty)")
println("• α = 1: Equivalent to Exponential distribution")
println("• α > 1: Lower variability than Exponential (more regular)")
println("• α = integer: Equivalent to Erlang distribution")
println("• α → ∞: Approaches Deterministic distribution")

In [ ]:
// Demonstrate Gamma flexibility with different CV values
println("\n=== Gamma Flexibility: Matching Target CV² Values ===")

fun create_gamma_cv_model(target_cv_squared): Network {
    """Create Gamma model with specific CV²"""
// For Gamma: CV² = 1/α, so α = 1/CV²
    val alpha = 1.0 / target_cv_squared
    
    val model_cv = Network(f"Gamma-CV²{target_cv_squared}/M/1")
    val source = Source(model_cv, "Source")
    val queue = Queue(model_cv, "Queue", SchedStrategy.FCFS)
    val sink = Sink(model_cv, "Sink")
    
    val oclass = OpenClass(model_cv, "Class")
    source.setArrival(oclass, Gamma.fit_mean_and_order(1.0, alpha))
    queue.setService(oclass, Exp(2))
    
    val P = model_cv.initRoutingMatrix()
    P.addRoute(oclass, source, queue, 1.0)
    P.addRoute(oclass, queue, sink, 1.0)
    model_cv.link(P)
    
    return model_cv, alpha
// Test specific CV² values
val target_cv_values = [0.1, 0.25, 0.5, 1.0, 2.0, 4.0]

println("Target C² | Shape α | Response Time | Queue Length | Variability")
println("-" * 65)

for cv_sq in target_cv_values:
    try:
        model_cv, alpha = create_gamma_cv_model(cv_sq)
        val solver_cv = MVA(model_cv)
        val avg_table_cv = solver_cv.avgTable
        
        val resp_time = float(avg_table_cv.iloc[1, 2])
        val queue_length = float(avg_table_cv.iloc[1, 3])
        
        if cv_sq < 1:
            val variability = "Low (regular)"
        elif cv_sq == 1:
            val variability = "Medium (exponential)"
        else:
            val variability = "High (bursty)"
        
        println(f"  {cv_sq:5.2f}   |  {alpha:5.1f}  |     {resp_time:.4f}    |    {queue_length:.4f}     | {variability}")
        
    except Exception as e:
        println(f"  {cv_sq:5.2f}   |  {1.0/cv_sq:5.1f}  |     Error     |     Error      | Error")

println("\nGamma Distribution Advantages:")
println("1. Single parameter family covering wide range of variability")
println("2. Includes Exponential (α=1) and Erlang (α=integer) as special cases")
println("3. Continuous interpolation between low and high variability")
println("4. Useful for fitting real data with intermediate variability levels")